Primero importo las librerias

In [ ]:
import json
import cv2
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.model_selection import train_test_split
usuario=2
import os
from matplotlib.image import imread
import random
import numpy as np
import glob as glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Vamos a mejorar yolov5l

In [ ]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16575, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 16575 (delta 28), reused 37 (delta 18), pack-reused 16522
Receiving objects: 100% (16575/16575), 15.09 MiB | 28.66 MiB/s, done.
Resolving deltas: 100% (11381/11381), done.


In [ ]:
%cd yolov5/
!pwd
!pip install -r requirements.txt

/content/yolov5/yolov5
/content/yolov5/yolov5


In [ ]:
import os

def verify_labels(images_path, labels_path):
    # Listar los archivos en cada directorio
    image_files = set(os.listdir(images_path))
    label_files = set(os.listdir(labels_path))

    # Convertir los nombres de los archivos de imágenes a formato de etiquetas si es necesario
    image_files = {file.replace('.jpg', '.txt').replace('.png', '.txt') for file in image_files}

    # Verificar que cada imagen tenga su correspondiente archivo de etiqueta
    missing_labels = image_files - label_files
    extra_labels = label_files - image_files

    if not missing_labels and not extra_labels:
        print("Todos los archivos de imagen tienen etiquetas correspondientes y viceversa.")
    else:
        if missing_labels:
            print("Faltan etiquetas para los siguientes archivos de imagen:", missing_labels)
        if extra_labels:
            print("Hay etiquetas extra que no corresponden a archivos de imagen:", extra_labels)

# Rutas a las carpetas
train_images_path = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images'
train_labels_path = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/labels'
test_images_path = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Train/images'
test_labels_path = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Train/labels'

# Verificación para entrenamiento y prueba
verify_labels(train_images_path, train_labels_path)
verify_labels(test_images_path, test_labels_path)


Todos los archivos de imagen tienen etiquetas correspondientes y viceversa.
Todos los archivos de imagen tienen etiquetas correspondientes y viceversa.


In [ ]:
#yaml sofi
yaml_content ="""
   path: '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido'
   train: '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Train/images' # Subruta a tus imágenes de entrenamiento
   val: '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images' # Subruta a tus imágenes de validación
   nc: 1  # Número de clases
   names: ['plant']  # Nombres de las clases
   """

#yaml_content ="""
 # path: '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test_Reducido'
  #train: '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test_Reducido/Train/images' # Subruta a tus imágenes de entrenamiento
  #val: '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test_Reducido/Test/images' # Subruta a tus imágenes de validación
  #nc: 1  # Número de clases
  #names: ['plant']  # Nombres de las clases
  #"""

file_path= "/content/drive/MyDrive/Proyecto final/Training Sofi/dataSofiAzu.yaml"

#file_path= "/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Training Sofi/dataSofiAzu.yaml"

with open(file_path, 'w') as file:
    file.write(yaml_content)

Vamos a probas la optimizacion de hiperparametros

DATA AUGMENTATION

In [ ]:
!pip install albumentations

  Using cached Pillow-10.0.1-cp310-cp310-manylinux_2_28_x86_64.whl (3.6 MB)
  Attempting uninstall: pillow
    Found existing installation: pillow 10.3.0
    Uninstalling pillow-10.3.0:
      Successfully uninstalled pillow-10.3.0


In [ ]:
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


Definamos las transformaciones

In [ ]:
import os

input_folder_path = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Train/images'
output_folder = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Train/imagesModified'

#input_folder_path = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test_Reducido/Train/images'
#output_folder = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test_Reducido/Train/imagesModified'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
transform = A.Compose([
    A.OneOf([
        A.Rotate(limit=40, p=1),  # Rotar hasta 40 grados
        A.HorizontalFlip(p=1),  # Volteo horizontal
        A.RandomBrightnessContrast(p=1),  # Ajuste de brillo y contraste
        A.GaussNoise(p=1),  # Inyección de ruido
        A.Perspective(p=1),  # Transformación de perspectiva
    ], p=0.05)  # Probabilidad del 5% de aplicar una transformación
])

NameError: name 'A' is not defined

Cargar y transformar imágenes de la carpeta Train

In [ ]:
mages = os.listdir(input_folder_path)

for img_name in images:
    img_path = os.path.join(input_folder_path, img_name)
    image = np.array(Image.open(img_path))
    original_image = image.copy()  # Copia de la imagen original para visualización

    # Aplicar transformación con una probabilidad del 5%
    if random.random() < 0.05:
        augmented = transform(image=image)  # Aplicar transformación
        transformed_image = augmented['image']

        if not np.array_equal(original_image, transformed_image):
            # Guardar imagen transformada en la carpeta de salida
            output_path = os.path.join(output_folder, img_name)
            cv2.imwrite(output_path, cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR))
            print(f"Saved transformed image to {output_path}")

            # Opcional: Mostrar ambas imágenes
            plt.figure(figsize=(12, 6))
            plt.subplot(1, 2, 1)
            plt.title('Original')
            plt.imshow(original_image)
            plt.subplot(1, 2, 2)
            plt.title('Transformed')
            plt.imshow(transformed_image)
            plt.show()

Pruebo con tamaño 832 y batch size 16

In [ ]:
!python "/content/drive/MyDrive/Proyecto final/yolov5/train.py" --data "/content/drive/MyDrive/Proyecto final/Training Sofi/dataSofiAzu.yaml" --weights yolov5s.pt --img 832  --epochs 30 --batch-size 16 --name "/content/drive/MyDrive/Proyecto final/Training Sofi/ochotreintaydos"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.4/756.4 kB 13.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

Pruebo con tamaño 800 y Batch size 16

In [ ]:
!python "/content/drive/MyDrive/Proyecto final/yolov5/train.py" --data "/content/drive/MyDrive/Proyecto final/Training Sofi/dataSofiAzu.yaml" --weights yolov5s.pt --img 640  --epochs 30 --batch-size 10 --name "/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez"

2024-05-04 22:02:22.610584: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 22:02:22.610639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 22:02:22.612268: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/Proyecto final/Training Sofi/dataSofiAzu.yaml, hyp=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=10, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=Non

Veamos en que aspectos esta fallando el modelo

In [ ]:
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt' --img 640 --conf 0.25 --source '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detect' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt'], source=/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detect, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/content/drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto': No such file or directory
YOLOv5 🚀 2024-5-1 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.55

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels
def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_pred = 0
    total_real = 0

    for filename, real_boxes in real_labels.items():
        pred_boxes = pred_labels.get(filename, [])
        total_pred += len(pred_boxes)
        total_real += len(real_boxes)

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        print(f'Image: {filename}')
        print(f'Predicted objects: {len(pred_boxes)}, Real objects: {len(real_boxes)}')

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}')
    print(f'Total Predicted: {total_pred}, Total Real: {total_real}')

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detect/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/labels')

# Calcula métricas
calculate_metrics(pred_labels, real_labels)


Image: 53994_6-6-1-4.txt
Predicted objects: 27, Real objects: 20
Image: 3258_2-3-1-4.txt
Predicted objects: 48, Real objects: 45
Image: 54118_6-9-4-1.txt
Predicted objects: 40, Real objects: 31
Image: 3181_76-6-1-1.txt
Predicted objects: 23, Real objects: 15
Image: 34234_8-2-1-5.txt
Predicted objects: 43, Real objects: 88
Image: 37420_2-1-1-5.txt
Predicted objects: 26, Real objects: 24
Image: 54090_31-14-1-4.txt
Predicted objects: 14, Real objects: 9
Image: 38028_1-6-16-4.txt
Predicted objects: 20, Real objects: 18
Image: 53851_29-8-1-4.txt
Predicted objects: 34, Real objects: 20
Image: 33881_1-2-1-8.txt
Predicted objects: 47, Real objects: 36
Image: 35656_7-3-1-8.txt
Predicted objects: 37, Real objects: 51
Image: 3152_8-2-1-3.txt
Predicted objects: 18, Real objects: 15
Image: 3153_35-2-1-4.txt
Predicted objects: 17, Real objects: 15
Image: 5453_5-5-1-8.txt
Predicted objects: 50, Real objects: 31
Image: 5702_28-1-1-1.txt
Predicted objects: 27, Real objects: 21
Image: 3707_13-3-1-3.txt


In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.3):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detect/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.6623, Recall: 0.7404, F1-Score: 0.6992

Métricas por imagen:
Image: 53994_6-6-1-4.txt, Precision: 0.5926, Recall: 0.8000, F1-Score: 0.6809
Image: 3258_2-3-1-4.txt, Precision: 0.7292, Recall: 0.7778, F1-Score: 0.7527
Image: 54118_6-9-4-1.txt, Precision: 0.6750, Recall: 0.8710, F1-Score: 0.7606
Image: 3181_76-6-1-1.txt, Precision: 0.6522, Recall: 1.0000, F1-Score: 0.7895
Image: 34234_8-2-1-5.txt, Precision: 0.9767, Recall: 0.4773, F1-Score: 0.6412
Image: 37420_2-1-1-5.txt, Precision: 0.8846, Recall: 0.9583, F1-Score: 0.9200
Image: 54090_31-14-1-4.txt, Precision: 0.5714, Recall: 0.8889, F1-Score: 0.6957
Image: 38028_1-6-16-4.txt, Precision: 0.2500, Recall: 0.2778, F1-Score: 0.2632
Image: 53851_29-8-1-4.txt, Precision: 0.5294, Recall: 0.9000, F1-Score: 0.6667
Image: 33881_1-2-1-8.txt, Precision: 0.7660, Recall: 1.0000, F1-Score: 0.8675
Image: 35656_7-3-1-8.txt, Precision: 0.4324, Recall: 0.3137, F1-Score: 0.3636
Image: 3152_8-2-1-3.txt, Precision: 0.7778, R

In [1]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/labels'


# draw_yolo_annotations(image_folder, label_folder)

In [2]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detect/labels'

# draw_yolo_annotations(image_folder, label_folder)

OCHOTRINTADIEZ

In [ ]:
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/ochotreintaydos/weights/best.pt' --img 830 --source '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/OCHOTRINTAYDOS/detect' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/Proyecto final/Training Sofi/ochotreintaydos/weights/best.pt'], source=/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images, data=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/coco128.yaml, imgsz=[830, 830], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/runs/detect, name=/content/drive/MyDrive/Proyecto final/Training Sofi/OCHOTRINTAYDOS/detect, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/content/drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto': No such file or directory
YOLOv5 🚀 2024-5-1 Python-3.10.1

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.3):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/OCHOTRINTAYDOS/detect/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.7552, Recall: 0.7632, F1-Score: 0.7592

Métricas por imagen:
Image: 53994_6-6-1-4.txt, Precision: 0.5517, Recall: 0.8000, F1-Score: 0.6531
Image: 3258_2-3-1-4.txt, Precision: 0.8222, Recall: 0.8222, F1-Score: 0.8222
Image: 54118_6-9-4-1.txt, Precision: 0.7500, Recall: 0.8710, F1-Score: 0.8060
Image: 3181_76-6-1-1.txt, Precision: 0.8333, Recall: 1.0000, F1-Score: 0.9091
Image: 34234_8-2-1-5.txt, Precision: 0.9231, Recall: 0.5455, F1-Score: 0.6857
Image: 37420_2-1-1-5.txt, Precision: 0.8889, Recall: 1.0000, F1-Score: 0.9412
Image: 54090_31-14-1-4.txt, Precision: 0.6667, Recall: 0.8889, F1-Score: 0.7619
Image: 38028_1-6-16-4.txt, Precision: 0.3125, Recall: 0.2778, F1-Score: 0.2941
Image: 53851_29-8-1-4.txt, Precision: 0.6667, Recall: 0.9000, F1-Score: 0.7660
Image: 33881_1-2-1-8.txt, Precision: 0.9474, Recall: 1.0000, F1-Score: 0.9730
Image: 35656_7-3-1-8.txt, Precision: 0.6170, Recall: 0.5686, F1-Score: 0.5918
Image: 3152_8-2-1-3.txt, Precision: 1.0000, R

In [ ]:
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/weights/best.pt' --img 640 --conf 0.25 --source '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detect' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/weights/best.pt'], source=/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images, data=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/runs/detect, name=/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detect, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/content/drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto': No such file or directory
YOLOv5 🚀 2024-5-1 Python-3.10.12 torch-2.2.1+c

Entreno de nuevo teniendo en cuenta mis resultados y ajustando los hiperparametros

In [ ]:
!python "/content/drive/MyDrive/Proyecto final/yolov5/train.py" --data "/content/drive/MyDrive/Proyecto final/Training Sofi/dataSofiAzu.yaml" --weights yolov5s.pt --img 640  --epochs 50 --batch-size 10 --name "/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo"

2024-05-05 19:07:51.411205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 19:07:51.411253: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 19:07:51.412744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/Proyecto final/Training Sofi/dataSofiAzu.yaml, hyp=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=10, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=Non

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels
def calculate_metrics(pred_labels, real_labels, iou_threshold=0.3):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_pred = 0
    total_real = 0

    for filename, real_boxes in real_labels.items():
        pred_boxes = pred_labels.get(filename, [])
        total_pred += len(pred_boxes)
        total_real += len(real_boxes)

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        print(f'Image: {filename}')
        print(f'Predicted objects: {len(pred_boxes)}, Real objects: {len(real_boxes)}')

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}')
    print(f'Total Predicted: {total_pred}, Total Real: {total_real}')

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detect/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/labels')

# Calcula métricas
calculate_metrics(pred_labels, real_labels)


Image: 53994_6-6-1-4.txt
Predicted objects: 25, Real objects: 20
Image: 3258_2-3-1-4.txt
Predicted objects: 53, Real objects: 45
Image: 54118_6-9-4-1.txt
Predicted objects: 36, Real objects: 31
Image: 3181_76-6-1-1.txt
Predicted objects: 16, Real objects: 15
Image: 34234_8-2-1-5.txt
Predicted objects: 43, Real objects: 88
Image: 37420_2-1-1-5.txt
Predicted objects: 24, Real objects: 24
Image: 54090_31-14-1-4.txt
Predicted objects: 13, Real objects: 9
Image: 38028_1-6-16-4.txt
Predicted objects: 17, Real objects: 18
Image: 53851_29-8-1-4.txt
Predicted objects: 30, Real objects: 20
Image: 33881_1-2-1-8.txt
Predicted objects: 43, Real objects: 36
Image: 35656_7-3-1-8.txt
Predicted objects: 40, Real objects: 51
Image: 3152_8-2-1-3.txt
Predicted objects: 17, Real objects: 15
Image: 3153_35-2-1-4.txt
Predicted objects: 15, Real objects: 15
Image: 5453_5-5-1-8.txt
Predicted objects: 46, Real objects: 31
Image: 5702_28-1-1-1.txt
Predicted objects: 24, Real objects: 21
Image: 3707_13-3-1-3.txt


In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.3):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detect/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.7403, Recall: 0.7608, F1-Score: 0.7504

Métricas por imagen:
Image: 53994_6-6-1-4.txt, Precision: 0.7200, Recall: 0.9000, F1-Score: 0.8000
Image: 3258_2-3-1-4.txt, Precision: 0.7547, Recall: 0.8889, F1-Score: 0.8163
Image: 54118_6-9-4-1.txt, Precision: 0.7500, Recall: 0.8710, F1-Score: 0.8060
Image: 3181_76-6-1-1.txt, Precision: 0.9375, Recall: 1.0000, F1-Score: 0.9677
Image: 34234_8-2-1-5.txt, Precision: 0.9302, Recall: 0.4545, F1-Score: 0.6107
Image: 37420_2-1-1-5.txt, Precision: 0.9583, Recall: 0.9583, F1-Score: 0.9583
Image: 54090_31-14-1-4.txt, Precision: 0.6154, Recall: 0.8889, F1-Score: 0.7273
Image: 38028_1-6-16-4.txt, Precision: 0.4118, Recall: 0.3889, F1-Score: 0.4000
Image: 53851_29-8-1-4.txt, Precision: 0.6000, Recall: 0.9000, F1-Score: 0.7200
Image: 33881_1-2-1-8.txt, Precision: 0.8372, Recall: 1.0000, F1-Score: 0.9114
Image: 35656_7-3-1-8.txt, Precision: 0.5500, Recall: 0.4314, F1-Score: 0.4835
Image: 3152_8-2-1-3.txt, Precision: 0.8824, R

In [3]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detect/labels'

# draw_yolo_annotations(image_folder, label_folder)

CUAL GENERALIZA MEJOR?

NOMBRE DE DETECCION: detect2
NOMBRE DEL MODELO: SEISCUARENTADIEZ
PROBEMOS EL PRIMER MODELO CON CONFIANZA DE 0.25 Y  SIN iou-thres

In [ ]:
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt' --img 640 --conf 0.25 --source '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detect' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt'], source=/content/drive/MyDrive/Proyecto final/Train_Test/Test/images, data=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/runs/detect, name=/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detect, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/content/drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto': No such file or directory
YOLOv5 🚀 2024-5-1 Python-3.10.12 torc

NOMBRE DE DETECCION: detect2
NOMBRE DEL MODELO: ULTIMO2
PROBEMOS EL PRIMER MODELO CON CONFIANZA DE 0.25 Y SIN iou-thres

In [ ]:
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/weights/best.pt' --img 640 --conf 0.25 --source '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detect' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/weights/best.pt'], source=/content/drive/MyDrive/Proyecto final/Train_Test/Test/images, data=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/runs/detect, name=/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detect, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/content/drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto': No such file or directory
YOLOv5 🚀 2024-5-1 Python-3.10.12 torch-2.2.1+cu121 CUDA

In [ ]:
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/weights/best.pt' --img 640 --conf 0.25 --source '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detectCHICO0.5' --save-txt --save-conf


NOMBRE DE DETECCION: detectultimo
NOMBRE DEL MODELO: SEISCUARENTADIEZ
PROBEMOS EL PRIMER MODELO CON CONFIANZA DE 0.25 Y  iou-thres DE 0.4

In [ ]:
!pip install ultralytics

!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt' --img 640 --conf 0.25 --iou-thres 0.40 --source '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt'], source=/content/drive/MyDrive/Proyecto final/Train_Test/Test/images, data=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.4, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/runs/detect, name=/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0'] not found, attempting AutoUpdate...
ERROR: pip's dependency resolver does not currently 

In [ ]:
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt' --img 640 --conf 0.25 --iou-thres 0.40 --source '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo' --save-txt --save-conf


NOMBRE DE DETECCION: detectultimo0.3
NOMBRE DEL MODELO: SEISCUARENTADIEZ
PROBEMOS EL PRIMER MODELO CON CONFIANZA DE 0.25 Y  iou-thres DE 0.3

In [ ]:
!pip install ultralytics
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt' --img 640 --conf 0.25 --source '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/FINALdetectultimo0.5' --save-txt --save-conf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.6/756.6 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:

!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt' --img 640 --conf 0.25 --iou-thres 0.40 --source '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/FINALdetectultimo0.4' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt'], source=/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images, data=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.4, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/runs/detect, name=/content/drive/MyDrive/Proyecto final/Training Sofi/FINALdetectultimo0.4, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/content/drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto': No such file or directory
YOLOv5 🚀 2024-5-1 Python-3.10.12

In [ ]:
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt' --img 640 --conf 0.25 --iou-thres 0.30 --source '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/FINALdetectultimo0.3' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt'], source=/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images, data=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.3, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/runs/detect, name=/content/drive/MyDrive/Proyecto final/Training Sofi/FINALdetectultimo0.3, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/content/drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto': No such file or directory
YOLOv5 🚀 2024-5-1 Python-3.10.12

In [ ]:
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt' --img 640 --conf 0.25 --iou-thres 0.20 --source '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/FINALdetectultimo0.2' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt'], source=/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images, data=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.2, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/runs/detect, name=/content/drive/MyDrive/Proyecto final/Training Sofi/FINALdetectultimo0.2, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/content/drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto': No such file or directory
YOLOv5 🚀 2024-5-1 Python-3.10.12

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.3):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/FINALdetectultimo0.2/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.8002, Recall: 0.7201, F1-Score: 0.7580

Métricas por imagen:
Image: 53994_6-6-1-4.txt, Precision: 0.6957, Recall: 0.8000, F1-Score: 0.7442
Image: 3258_2-3-1-4.txt, Precision: 0.8000, Recall: 0.7111, F1-Score: 0.7529
Image: 54118_6-9-4-1.txt, Precision: 0.8710, Recall: 0.8710, F1-Score: 0.8710
Image: 3181_76-6-1-1.txt, Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000
Image: 34234_8-2-1-5.txt, Precision: 0.9756, Recall: 0.4545, F1-Score: 0.6202
Image: 37420_2-1-1-5.txt, Precision: 0.9583, Recall: 0.9583, F1-Score: 0.9583
Image: 54090_31-14-1-4.txt, Precision: 0.7000, Recall: 0.7778, F1-Score: 0.7368
Image: 38028_1-6-16-4.txt, Precision: 0.2778, Recall: 0.2778, F1-Score: 0.2778
Image: 53851_29-8-1-4.txt, Precision: 0.6923, Recall: 0.9000, F1-Score: 0.7826
Image: 33881_1-2-1-8.txt, Precision: 0.9730, Recall: 1.0000, F1-Score: 0.9863
Image: 35656_7-3-1-8.txt, Precision: 0.4444, Recall: 0.3137, F1-Score: 0.3678
Image: 3152_8-2-1-3.txt, Precision: 0.9333, R

In [4]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Training Sofi/FINALdetectultimo0.2/labels'

# draw_yolo_annotations(image_folder, label_folder)

NOMBRE DE DETECCION: detectultimo0.2
NOMBRE DEL MODELO: SEISCUARENTADIEZ
PROBEMOS EL PRIMER MODELO CON CONFIANZA DE 0.25 Y  iou-thres DE 0.2

In [ ]:
!python '/content/drive/MyDrive/Proyecto final/yolov5/detect.py' --weights '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt' --img 640 --conf 0.25 --iou-thres 0.20 --source '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images' --name '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo0.2' --save-txt --save-conf


detect: weights=['/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/weights/best.pt'], source=/content/drive/MyDrive/Proyecto final/Train_Test/Test/images, data=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.2, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto final/yolov5/runs/detect, name=/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo0.2, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/content/drive/.shortcut-targets-by-id/1CAu6a2qYFZGqMFQXQFZdHKPVaysMB2xO/Proyecto': No such file or directory
YOLOv5 🚀 2024-5-1 Python-3.10

Modelo 1 clasico

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.25):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detect2/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.4537, Recall: 0.5380, F1-Score: 0.4922

Métricas por imagen:
Image: 3175_1-5-1-3.txt, Precision: 0.3810, Recall: 0.4444, F1-Score: 0.4103
Image: 53819_3-10-1-4.txt, Precision: 0.2083, Recall: 0.2941, F1-Score: 0.2439
Image: 53823_2-4-2-1.txt, Precision: 0.3750, Recall: 0.4737, F1-Score: 0.4186
Image: 53853_37-18-1-1.txt, Precision: 0.1818, Recall: 0.2667, F1-Score: 0.2162
Image: 37186_8-9-1-5.txt, Precision: 0.1250, Recall: 0.1500, F1-Score: 0.1364
Image: 53994_6-8-1-1.txt, Precision: 0.5172, Recall: 0.6250, F1-Score: 0.5660
Image: 3140_80-9-1-2.txt, Precision: 0.7500, Recall: 0.8182, F1-Score: 0.7826
Image: 3153_39-2-1-2.txt, Precision: 0.2778, Recall: 0.3125, F1-Score: 0.2941
Image: 5702_29-1-1-3.txt, Precision: 0.6500, Recall: 0.7647, F1-Score: 0.7027
Image: 34417_15-3-1-1.txt, Precision: 0.6327, Recall: 0.7949, F1-Score: 0.7045
Image: 38028_1-6-11-12.txt, Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000
Image: 37807_1-7-15-4.txt, Precision: 0.700

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels
def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_pred = 0
    total_real = 0

    for filename, real_boxes in real_labels.items():
        pred_boxes = pred_labels.get(filename, [])
        total_pred += len(pred_boxes)
        total_real += len(real_boxes)

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        print(f'Image: {filename}')
        print(f'Predicted objects: {len(pred_boxes)}, Real objects: {len(real_boxes)}')

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}')
    print(f'Total Predicted: {total_pred}, Total Real: {total_real}')

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detect2/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels')

# Calcula métricas
calculate_metrics(pred_labels, real_labels)


In [ ]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detect2/labels'

draw_yolo_annotations(image_folder, label_folder)

Modelo 1 0.4

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.4906, Recall: 0.5301, F1-Score: 0.5096

Métricas por imagen:
Image: 3175_1-5-1-3.txt, Precision: 0.4211, Recall: 0.4444, F1-Score: 0.4324
Image: 53819_3-10-1-4.txt, Precision: 0.2381, Recall: 0.2941, F1-Score: 0.2632
Image: 53823_2-4-2-1.txt, Precision: 0.4091, Recall: 0.4737, F1-Score: 0.4390
Image: 53853_37-18-1-1.txt, Precision: 0.1818, Recall: 0.2667, F1-Score: 0.2162
Image: 37186_8-9-1-5.txt, Precision: 0.1395, Recall: 0.1500, F1-Score: 0.1446
Image: 53994_6-8-1-1.txt, Precision: 0.5357, Recall: 0.6250, F1-Score: 0.5769
Image: 3140_80-9-1-2.txt, Precision: 0.8182, Recall: 0.8182, F1-Score: 0.8182
Image: 3153_39-2-1-2.txt, Precision: 0.2941, Recall: 0.3125, F1-Score: 0.3030
Image: 5702_29-1-1-3.txt, Precision: 0.7222, Recall: 0.7647, F1-Score: 0.7429
Image: 34417_15-3-1-1.txt, Precision: 0.7045, Recall: 0.7949, F1-Score: 0.7470
Image: 38028_1-6-11-12.txt, Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000
Image: 37807_1-7-15-4.txt, Precision: 0.700

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels
def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_pred = 0
    total_real = 0

    for filename, real_boxes in real_labels.items():
        pred_boxes = pred_labels.get(filename, [])
        total_pred += len(pred_boxes)
        total_real += len(real_boxes)

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        print(f'Image: {filename}')
        print(f'Predicted objects: {len(pred_boxes)}, Real objects: {len(real_boxes)}')

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}')
    print(f'Total Predicted: {total_pred}, Total Real: {total_real}')

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels')

# Calcula métricas
calculate_metrics(pred_labels, real_labels)


In [ ]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detectultimo/labels'

draw_yolo_annotations(image_folder, label_folder)

Modelo 1 0.3

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo0.3/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.5269, Recall: 0.5235, F1-Score: 0.5252

Métricas por imagen:
Image: 3175_1-5-1-3.txt, Precision: 0.4444, Recall: 0.4444, F1-Score: 0.4444
Image: 53819_3-10-1-4.txt, Precision: 0.2105, Recall: 0.2353, F1-Score: 0.2222
Image: 53823_2-4-2-1.txt, Precision: 0.3684, Recall: 0.3684, F1-Score: 0.3684
Image: 53853_37-18-1-1.txt, Precision: 0.2353, Recall: 0.2667, F1-Score: 0.2500
Image: 37186_8-9-1-5.txt, Precision: 0.1500, Recall: 0.1500, F1-Score: 0.1500
Image: 53994_6-8-1-1.txt, Precision: 0.5556, Recall: 0.6250, F1-Score: 0.5882
Image: 3140_80-9-1-2.txt, Precision: 0.9000, Recall: 0.8182, F1-Score: 0.8571
Image: 3153_39-2-1-2.txt, Precision: 0.3125, Recall: 0.3125, F1-Score: 0.3125
Image: 5702_29-1-1-3.txt, Precision: 0.7647, Recall: 0.7647, F1-Score: 0.7647
Image: 34417_15-3-1-1.txt, Precision: 0.7949, Recall: 0.7949, F1-Score: 0.7949
Image: 38028_1-6-11-12.txt, Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000
Image: 37807_1-7-15-4.txt, Precision: 0.714

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.30):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/AUGMENTED/Detect/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test_Reducido/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.7945, Recall: 0.6607, F1-Score: 0.7215

Métricas por imagen:
Image: 53994_6-6-1-4.txt, Precision: 0.6190, Recall: 0.6500, F1-Score: 0.6341
Image: 3258_2-3-1-4.txt, Precision: 0.7647, Recall: 0.5778, F1-Score: 0.6582
Image: 54118_6-9-4-1.txt, Precision: 0.8929, Recall: 0.8065, F1-Score: 0.8475
Image: 3181_76-6-1-1.txt, Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000
Image: 34234_8-2-1-5.txt, Precision: 0.8529, Recall: 0.3295, F1-Score: 0.4754
Image: 37420_2-1-1-5.txt, Precision: 0.9231, Recall: 1.0000, F1-Score: 0.9600
Image: 54090_31-14-1-4.txt, Precision: 0.6000, Recall: 0.6667, F1-Score: 0.6316
Image: 38028_1-6-16-4.txt, Precision: 0.1765, Recall: 0.1667, F1-Score: 0.1714
Image: 53851_29-8-1-4.txt, Precision: 0.7037, Recall: 0.9500, F1-Score: 0.8085
Image: 33881_1-2-1-8.txt, Precision: 0.9474, Recall: 1.0000, F1-Score: 0.9730
Image: 35656_7-3-1-8.txt, Precision: 0.4242, Recall: 0.2745, F1-Score: 0.3333
Image: 3152_8-2-1-3.txt, Precision: 1.0000, R

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels
def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_pred = 0
    total_real = 0

    for filename, real_boxes in real_labels.items():
        pred_boxes = pred_labels.get(filename, [])
        total_pred += len(pred_boxes)
        total_real += len(real_boxes)

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        print(f'Image: {filename}')
        print(f'Predicted objects: {len(pred_boxes)}, Real objects: {len(real_boxes)}')

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}')
    print(f'Total Predicted: {total_pred}, Total Real: {total_real}')

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo0.3/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels')

# Calcula métricas
calculate_metrics(pred_labels, real_labels)


Image: 3175_1-5-1-3.txt
Predicted objects: 18, Real objects: 18
Image: 53819_3-10-1-4.txt
Predicted objects: 38, Real objects: 34
Image: 53823_2-4-2-1.txt
Predicted objects: 19, Real objects: 19
Image: 53853_37-18-1-1.txt
Predicted objects: 17, Real objects: 15
Image: 37186_8-9-1-5.txt
Predicted objects: 40, Real objects: 40
Image: 53994_6-8-1-1.txt
Predicted objects: 27, Real objects: 24
Image: 3140_80-9-1-2.txt
Predicted objects: 10, Real objects: 11
Image: 3153_39-2-1-2.txt
Predicted objects: 16, Real objects: 16
Image: 5702_29-1-1-3.txt
Predicted objects: 17, Real objects: 17
Image: 34417_15-3-1-1.txt
Predicted objects: 39, Real objects: 39
Image: 38028_1-6-11-12.txt
Predicted objects: 18, Real objects: 18
Image: 37807_1-7-15-4.txt
Predicted objects: 28, Real objects: 24
Image: 37801_16-4-1-3.txt
Predicted objects: 51, Real objects: 92
Image: 53819_7-12-1-4.txt
Predicted objects: 36, Real objects: 28
Image: 6672_42-4-1-6.txt
Predicted objects: 16, Real objects: 15
Image: 37326_16-2

In [ ]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo0.3/labels'

draw_yolo_annotations(image_folder, label_folder)

Modelo 1 0.2

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo0.2/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.5361, Recall: 0.5166, F1-Score: 0.5262

Métricas por imagen:
Image: 3175_1-5-1-3.txt, Precision: 0.4444, Recall: 0.4444, F1-Score: 0.4444
Image: 53819_3-10-1-4.txt, Precision: 0.2162, Recall: 0.2353, F1-Score: 0.2254
Image: 53823_2-4-2-1.txt, Precision: 0.3529, Recall: 0.3158, F1-Score: 0.3333
Image: 53853_37-18-1-1.txt, Precision: 0.2500, Recall: 0.2667, F1-Score: 0.2581
Image: 37186_8-9-1-5.txt, Precision: 0.1579, Recall: 0.1500, F1-Score: 0.1538
Image: 53994_6-8-1-1.txt, Precision: 0.5600, Recall: 0.5833, F1-Score: 0.5714
Image: 3140_80-9-1-2.txt, Precision: 0.9000, Recall: 0.8182, F1-Score: 0.8571
Image: 3153_39-2-1-2.txt, Precision: 0.3125, Recall: 0.3125, F1-Score: 0.3125
Image: 5702_29-1-1-3.txt, Precision: 0.7647, Recall: 0.7647, F1-Score: 0.7647
Image: 34417_15-3-1-1.txt, Precision: 0.7949, Recall: 0.7949, F1-Score: 0.7949
Image: 38028_1-6-11-12.txt, Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000
Image: 37807_1-7-15-4.txt, Precision: 0.740

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels
def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_pred = 0
    total_real = 0

    for filename, real_boxes in real_labels.items():
        pred_boxes = pred_labels.get(filename, [])
        total_pred += len(pred_boxes)
        total_real += len(real_boxes)

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        print(f'Image: {filename}')
        print(f'Predicted objects: {len(pred_boxes)}, Real objects: {len(real_boxes)}')

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}')
    print(f'Total Predicted: {total_pred}, Total Real: {total_real}')

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo0.2/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels')

# Calcula métricas
calculate_metrics(pred_labels, real_labels)


Image: 3175_1-5-1-3.txt
Predicted objects: 18, Real objects: 18
Image: 53819_3-10-1-4.txt
Predicted objects: 37, Real objects: 34
Image: 53823_2-4-2-1.txt
Predicted objects: 17, Real objects: 19
Image: 53853_37-18-1-1.txt
Predicted objects: 16, Real objects: 15
Image: 37186_8-9-1-5.txt
Predicted objects: 38, Real objects: 40
Image: 53994_6-8-1-1.txt
Predicted objects: 25, Real objects: 24
Image: 3140_80-9-1-2.txt
Predicted objects: 10, Real objects: 11
Image: 3153_39-2-1-2.txt
Predicted objects: 16, Real objects: 16
Image: 5702_29-1-1-3.txt
Predicted objects: 17, Real objects: 17
Image: 34417_15-3-1-1.txt
Predicted objects: 39, Real objects: 39
Image: 38028_1-6-11-12.txt
Predicted objects: 18, Real objects: 18
Image: 37807_1-7-15-4.txt
Predicted objects: 27, Real objects: 24
Image: 37801_16-4-1-3.txt
Predicted objects: 46, Real objects: 92
Image: 53819_7-12-1-4.txt
Predicted objects: 32, Real objects: 28
Image: 6672_42-4-1-6.txt
Predicted objects: 16, Real objects: 15
Image: 37326_16-2

In [5]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Training Sofi/seiscuarentadiez/detectultimo0.2/labels'

# draw_yolo_annotations(image_folder, label_folder)

Modelo 2

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels

def iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convertir de formato centro a esquinas
    x1_min, y1_min = x1 - w1 / 2, y1 - h1 / 2
    x1_max, y1_max = x1 + w1 / 2, y1 + h1 / 2
    x2_min, y2_min = x2 - w2 / 2, y2 - h2 / 2
    x2_max, y2_max = x2 + w2 / 2, y2 + h2 / 2

    # Calcular la intersección
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    inter_area = max(inter_x_max - inter_x_min, 0) * max(inter_y_max - inter_y_min, 0)

    # Calcular la unión
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    metrics_per_image = []
    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for filename, real_boxes in real_labels.items():
        true_positives = 0
        false_positives = 0
        false_negatives = 0

        pred_boxes = pred_labels.get(filename, [])

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

        precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics_per_image.append({
            'Filename': filename,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1_score
        })

    # Métricas generales
    precision_general = total_true_positives / (total_true_positives + total_false_positives) if total_true_positives + total_false_positives > 0 else 0
    recall_general = total_true_positives / (total_true_positives + total_false_negatives) if total_true_positives + total_false_negatives > 0 else 0
    f1_score_general = 2 * (precision_general * recall_general) / (precision_general + recall_general) if precision_general + recall_general > 0 else 0

    general_metrics = {
        'Precision': precision_general,
        'Recall': recall_general,
        'F1-Score': f1_score_general
    }

    return metrics_per_image, general_metrics

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detect2/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels')

# Calcula métricas
metrics_per_image, general_metrics = calculate_metrics(pred_labels, real_labels)

# Imprime las métricas generales
print("Métricas generales:")
print(f"Precision: {general_metrics['Precision']:.4f}, Recall: {general_metrics['Recall']:.4f}, F1-Score: {general_metrics['F1-Score']:.4f}")
print()

# Imprime las métricas por imagen
print("Métricas por imagen:")
for metrics in metrics_per_image:
    print(f"Image: {metrics['Filename']}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, F1-Score: {metrics['F1-Score']:.4f}")


Métricas generales:
Precision: 0.4917, Recall: 0.5594, F1-Score: 0.5234

Métricas por imagen:
Image: 3175_1-5-1-3.txt, Precision: 0.3000, Recall: 0.3333, F1-Score: 0.3158
Image: 53819_3-10-1-4.txt, Precision: 0.2778, Recall: 0.2941, F1-Score: 0.2857
Image: 53823_2-4-2-1.txt, Precision: 0.4231, Recall: 0.5789, F1-Score: 0.4889
Image: 53853_37-18-1-1.txt, Precision: 0.0909, Recall: 0.1333, F1-Score: 0.1081
Image: 37186_8-9-1-5.txt, Precision: 0.3617, Recall: 0.4250, F1-Score: 0.3908
Image: 53994_6-8-1-1.txt, Precision: 0.5200, Recall: 0.5417, F1-Score: 0.5306
Image: 3140_80-9-1-2.txt, Precision: 0.7692, Recall: 0.9091, F1-Score: 0.8333
Image: 3153_39-2-1-2.txt, Precision: 0.2353, Recall: 0.2500, F1-Score: 0.2424
Image: 5702_29-1-1-3.txt, Precision: 0.5500, Recall: 0.6471, F1-Score: 0.5946
Image: 34417_15-3-1-1.txt, Precision: 0.7143, Recall: 0.7692, F1-Score: 0.7407
Image: 38028_1-6-11-12.txt, Precision: 0.1579, Recall: 0.1667, F1-Score: 0.1622
Image: 37807_1-7-15-4.txt, Precision: 0.733

In [ ]:
import os
import numpy as np

def load_labels(directory):
    labels = {}
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            # Excluir el primer valor (clase) y el último valor (score), almacenar solo las coordenadas
            boxes = [list(map(float, line.split()))[1:5] for line in file.readlines()]
            labels[filename] = boxes
    return labels
def calculate_metrics(pred_labels, real_labels, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_pred = 0
    total_real = 0

    for filename, real_boxes in real_labels.items():
        pred_boxes = pred_labels.get(filename, [])
        total_pred += len(pred_boxes)
        total_real += len(real_boxes)

        matched = [False] * len(real_boxes)

        for pred_box in pred_boxes:
            match_found = False
            for i, real_box in enumerate(real_boxes):
                if iou(pred_box, real_box) >= iou_threshold:
                    if not matched[i]:
                        true_positives += 1
                        matched[i] = True
                        match_found = True
                        break
            if not match_found:
                false_positives += 1

        false_negatives += matched.count(False)

        print(f'Image: {filename}')
        print(f'Predicted objects: {len(pred_boxes)}, Real objects: {len(real_boxes)}')

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}')
    print(f'Total Predicted: {total_pred}, Total Real: {total_real}')

# Carga de datos
pred_labels = load_labels('/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detect2/labels')
real_labels = load_labels('/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels')

# Calcula métricas
calculate_metrics(pred_labels, real_labels)


Image: 3175_1-5-1-3.txt
Predicted objects: 20, Real objects: 18
Image: 53819_3-10-1-4.txt
Predicted objects: 36, Real objects: 34
Image: 53823_2-4-2-1.txt
Predicted objects: 26, Real objects: 19
Image: 53853_37-18-1-1.txt
Predicted objects: 22, Real objects: 15
Image: 37186_8-9-1-5.txt
Predicted objects: 47, Real objects: 40
Image: 53994_6-8-1-1.txt
Predicted objects: 25, Real objects: 24
Image: 3140_80-9-1-2.txt
Predicted objects: 13, Real objects: 11
Image: 3153_39-2-1-2.txt
Predicted objects: 17, Real objects: 16
Image: 5702_29-1-1-3.txt
Predicted objects: 20, Real objects: 17
Image: 34417_15-3-1-1.txt
Predicted objects: 42, Real objects: 39
Image: 38028_1-6-11-12.txt
Predicted objects: 19, Real objects: 18
Image: 37807_1-7-15-4.txt
Predicted objects: 30, Real objects: 24
Image: 37801_16-4-1-3.txt
Predicted objects: 64, Real objects: 92
Image: 53819_7-12-1-4.txt
Predicted objects: 39, Real objects: 28
Image: 6672_42-4-1-6.txt
Predicted objects: 19, Real objects: 15
Image: 37326_16-2

In [6]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Training Sofi/ultimo2/detect2/labels'

# draw_yolo_annotations(image_folder, label_folder)

Labels originales

In [7]:
import cv2
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_folder, label_folder):
    # Lista todos los archivos en la carpeta de imágenes y anotaciones
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

    # Procesar cada archivo de imagen
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = f"{base_name}.txt"

        # Verifica que el archivo de etiquetas exista
        if label_file in label_files:
            image_path = os.path.join(image_folder, img_file)
            label_path = os.path.join(label_folder, label_file)

            # Carga la imagen
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error cargando la imagen {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Leer el archivo .txt con anotaciones YOLO y dibujarlas
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    x_center, y_center, w, h = map(float, parts[1:5])
                    x_center *= image.shape[1]
                    y_center *= image.shape[0]
                    w *= image.shape[1]
                    h *= image.shape[0]
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)

                    # Dibujar el rectángulo sobre la imagen
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Mostrar la imagen
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.title(img_file)
            plt.show()
        else:
            print(f"No se encontró archivo de etiquetas para {img_file}")

# Rutas a las carpetas de imágenes y etiquetas

image_folder = '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images'
label_folder = '/content/drive/MyDrive/Proyecto final/Train_Test/Test/labels'

# draw_yolo_annotations(image_folder, label_folder)

Entrenamos ahora si con el dataset grande

In [ ]:
#yaml sofi
yaml_content ="""
   path: '/content/drive/MyDrive/Proyecto final/Train_Test'
   train: '/content/drive/MyDrive/Proyecto final/Train_Test/Train/images' # Subruta a tus imágenes de entrenamiento
   val: '/content/drive/MyDrive/Proyecto final/Train_Test/Test/images' # Subruta a tus imágenes de validación
   nc: 1  # Número de clases
   names: ['plant']  # Nombres de las clases
   """

file_path= "/content/drive/MyDrive/Proyecto final/Training Sofi/datafinal.yaml"

#file_path= "/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Training Sofi/dataSofiAzu.yaml"

with open(file_path, 'w') as file:
    file.write(yaml_content)

In [ ]:
!python "/content/drive/MyDrive/Proyecto final/yolov5/train.py" --data "/content/drive/MyDrive/Proyecto final/Training Sofi/datafinal.yaml" --weights yolov5s.pt --img 640  --epochs 50 --batch-size 10 --name "/content/drive/MyDrive/Proyecto final/Training Sofi/TRAININGcompleto"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many